Imports

In [6]:
import psycopg
import os
from sentence_transformers import SentenceTransformer

c:\Users\dell\Desktop\AI project\Chatbot-RAG-Sofiene\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Database connection

In [7]:
conn = psycopg.connect(
    dbname="rag_chatbot",
    user="postgres",
    password="admin",  
    host="localhost",
    port=5432
)

cursor = conn.cursor()
print("Connexion établie avec succès.")

Connexion établie avec succès.


Create the table (ID, corpus TEXT, embedding FLOAT)

In [8]:
cursor.execute("""
    DROP TABLE IF EXISTS embeddings;
""")

cursor.execute("""
    CREATE TABLE embeddings (
        id SERIAL PRIMARY KEY,
        corpus TEXT,
        embedding vector(384)   -- pgvector: length must match your model
    );
""")

conn.commit()
print("Table 'embeddings' créée avec succès.")


Table 'embeddings' créée avec succès.


Load the corpus from /data/TRANS_TXT

In [9]:
corpus_folder = "../../data/TRANS_TXT"
corpus_list = []

for filename in os.listdir(corpus_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(corpus_folder, filename)

        # Try UTF-8, fallback to latin-1
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.read().split("\n")
        except UnicodeDecodeError:
            with open(file_path, "r", encoding="latin-1") as f:
                lines = f.read().split("\n")
        
        cleaned = [
            line.strip()
            for line in lines
            if line.strip() != "" and not line.strip().startswith("<")
        ]
        
        corpus_list.extend(cleaned)

print("Nombre de lignes du corpus :", len(corpus_list))
print(corpus_list[:10])


Nombre de lignes du corpus : 1062
['h: U B S bonjour', "c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e", 'h: je vous écoute', "c: c'est pour", "c: e c'est pour savoir si la fac pendant l'été e a des professeurs ou des des gens qui font des stages de de perfectionnement en anglais et en espagnol", 'h: e ce serait pour vous vous souhaiteriez', 'h: non', "c: non non c'est pas pour moi", 'c: ce serait pour ma fille', 'h: oui']


# Load SentenceTransformer model


In [10]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Modèle d'embedding chargé.")

def calculate_embedding(text: str):
    emb = model.encode(text).tolist()
    return emb

# --------------------------------------------
# Insert corpus + REAL embeddings
# --------------------------------------------
print("Insertion des embeddings dans PostgreSQL...")

for text in corpus_list:
    real_embedding = calculate_embedding(text)     # list of floats

    # Convert embedding list -> pgvector format: "[0.1, -0.24, ...]"
    emb_str = "[" + ",".join(str(x) for x in real_embedding) + "]"

    cursor.execute(
        "INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)",
        (text, emb_str)
    )

conn.commit()
print("Insertion du corpus terminée avec succès.")

Modèle d'embedding chargé.
Insertion des embeddings dans PostgreSQL...
Insertion du corpus terminée avec succès.


# Verify insertion


In [16]:
cursor.execute("SELECT id, corpus, embedding FROM embeddings LIMIT 5;")
rows = cursor.fetchall()
print(rows)


[(1, 'h: U B S bonjour', '[-0.05560039,0.08602783,0.040496986,-0.092654936,-0.020068293,0.028038695,0.092201814,0.02397672,0.050586537,0.024722686,0.031780243,-0.074718244,0.003305806,-0.07326118,-0.027850956,-0.07099355,-0.05988261,-0.020563569,-0.03914461,-0.04181432,-0.06351282,0.05533497,0.024358278,0.038584642,0.011354109,0.012282663,0.02288412,0.04067728,0.019387279,-0.077775404,-0.007601925,0.07224484,-0.0047157407,-0.032499216,-0.0032154038,0.0248458,0.023408258,-0.055935256,-0.01711195,-0.055043247,-0.055719513,-0.04185336,-0.0028921997,0.03636027,0.008702122,0.10497227,-0.039068036,0.041741643,-0.05230671,0.017404597,-0.09332428,0.021113385,0.01817345,0.040309522,0.056302223,0.087990135,-0.030075435,0.07442012,0.04314047,-0.09036795,-0.15077688,-0.014963142,0.007392623,-0.022195145,0.10181688,-0.013010515,-0.029680666,0.038862843,-0.0772376,0.106453046,0.02918139,-0.06191829,-0.07579011,0.017394817,-0.0070424257,-0.02269263,-0.052149475,-0.0466852,0.05484169,-6.9380556e-05,0.

In [17]:
cursor.execute("SELECT id, corpus, vector_dims(embedding) FROM embeddings LIMIT 5;")
print(cursor.fetchall())


[(1, 'h: U B S bonjour', 384), (2, "c: oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e", 384), (3, 'h: je vous écoute', 384), (4, "c: c'est pour", 384), (10, 'h: oui', 384)]


search_similar

In [18]:
def search_similar(query: str, top_k: int = 5):
    # 1️⃣ Convert question into embedding
    emb = calculate_embedding(query)

    # 2️⃣ Convert embedding to pgvector format
    emb_str = "[" + ",".join(str(x) for x in emb) + "]"

    # 3️⃣ Query PostgreSQL with vector similarity
    cursor.execute("""
        SELECT corpus, embedding <-> %s AS distance
        FROM embeddings
        ORDER BY embedding <-> %s
        LIMIT %s;
    """, (emb_str, emb_str, top_k))

    results = cursor.fetchall()
    return results


In [19]:
results = search_similar("bonjour je veux des informations")
for r in results:
    print(r[0], " | distance =", r[1])


c: e il travaille e au niveau informatique  | distance = 0.868477420445289
h: I U P sciences de gestion bonjour  | distance = 0.889852021055825
c: bonjour excusez-moi de vous déranger  | distance = 0.9112311961218429
c: e oui et puis les éventuellement la possibilité pour les consulter # sur sur internet ils apparaissent  | distance = 0.9112451939920497
c: oui bonjour est-ce que je pourrais avoir monsieur Nom s'il vous plaît  | distance = 0.9154052652983939


In [ ]:
from groq import Groq
import os

client = Groq(api_key="API_KEY")

def generate_answer(question, context):
    prompt = f"""
    CONTEXTE :
    {context}

    QUESTION :
    {question}

    REPONSE :
    """

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [37]:
def ask(question):
    docs = search_similar(question, top_k=5)
    context = "\n".join([d[0] for d in docs])
    answer = generate_answer(question, context)
    return answer

In [38]:
print(ask("Bonjour, je veux des informations sur les paiements électroniques"))


Bonjour ! Je serais ravi de vous fournir des informations sur les paiements électroniques. 

Nous allons avoir la réponse dès maintenant, il n'y a donc pas d'attente. 

Concernant les inscriptions, vous pouvez procéder en ligne sur notre site internet, où vous trouverez les formulaires nécessaires pour vous inscrire. Il vous faudra simplement fournir vos informations personnelles et vos données de paiement.

Pour les paiements électroniques, vous pouvez utiliser notre système sécurisé qui accepte les cartes de crédit et les prépaiements. Vous aurez la possibilité de consulter vos informations de paiement en ligne et de suivre vos transactions en temps réel.

Nous sommes là pour vous aider et répondre à vos questions, vous pouvez nous contacter à tout moment pour information ou assistance. 

N'hésitez pas à me poser d'autres questions si vous avez besoin de plus d'informations.


debug_ask

In [39]:
def debug_ask(question):
    # 1️⃣ Retrieve documents
    docs = search_similar(question, top_k=5)

    print("\n🔍 DOCUMENTS RETROUVÉS (triés par similarité):\n")
    for i, (text, dist) in enumerate(docs, start=1):
        print(f"📄 Document {i}:")
        print(f"Texte     : {text[:200]}{'...' if len(text)>200 else ''}")
        print(f"Distance  : {dist}")
        print("--------------------------------------------------")

    # 2️⃣ Build context
    context = "\n".join([d[0] for d in docs])

    print("\n📚 CONTEXTE ENVOYÉ AU MODÈLE :\n")
    print(context)
    print("\n--------------------------------------------------\n")

    # 3️⃣ Generate answer
    answer = generate_answer(question, context)

    print("💬 RÉPONSE DU MODÈLE :\n")
    print(answer)

    return answer


In [40]:
debug_ask("Bonjour, je veux des informations sur les paiements électroniques")



🔍 DOCUMENTS RETROUVÉS (triés par similarité):

📄 Document 1:
Texte     : c: quand est-ce qu'on aurait la réponse et comment se passent e les inscriptions e enfin le paiement
Distance  : 0.9906573172992785
--------------------------------------------------
📄 Document 2:
Texte     : h: de l'Information et de l'Orientation lui il doit peut-être e au au moins il vous donnera le numéro où téléphoner quoi
Distance  : 1.0241626761979243
--------------------------------------------------
📄 Document 3:
Texte     : h: e c'est combien de pages e
Distance  : 1.0290277987136756
--------------------------------------------------
📄 Document 4:
Texte     : c: e oui et puis les éventuellement la possibilité pour les consulter # sur sur internet ils apparaissent
Distance  : 1.0310319034106195
--------------------------------------------------
📄 Document 5:
Texte     : h: et ben alors on on on demande à ce moment là à Prénom4
Distance  : 1.0425079127681498
-----------------------------------------------

"Bonjour !\n\nJe voudrais vous informer que les paiements électroniques dépendent de plusieurs éléments, tels que la nature du paiement (achat en ligne, virement bancaire, etc.), la plateforme ou le site web utilisé pour la transaction et les informations fournies par l'organisme gestionnaire du paiement.\n\nEn général, voici les étapes pour les paiements électroniques :\n\n1. **Réponse à la question de la côte des coûts** : Les informations sur les paiements électroniques comprennent souvent les coûts associés aux transactions, qui peuvent varier en fonction de la méthode et de la plateforme utilisée.\n2. **Inspections préalables** : Avant de réaliser un paiement, il est recommandé de vérifier les informations du destinataire, le montant du paiement et les détails bancaires liés au paiement.\n3. **Paiement sécurisé** : Les paiements électroniques sont souvent sécurisés par des protocoles de cryptage (HTTPS/TLS) et les plateformes de paiement électronique utilisent des systèmes de sécu

# Close the connection


In [ ]:
cursor.close()
conn.close()
print("Connexion fermée.")